# Estrutura de Dados: Árvore Rubro-Negra com Persistência Parcial

## Descrição geral do trabalho:

O objetivo do trabalho é implementar uma árvore rubro-negra com persistência parcial seguindo o método descrito em sala. A entrada do programa é um arquivo de texto com várias operações sobre a estrutura (uma por linha) e a saída será um arquivo de texto com o resultado das operações (apenas daquelas cujo resultado pode ser impresso).

### Operações

#### Inclusão:

- Uma operação de inclusão será identificada por uma linha escrito `INC` seguido de um espaço e depois um inteiro. Este elemento deve ser incluído na estrutura de dados, uma nova versão criada e nada precisa ser impresso.

Exemplo de linha de inclusão:
```bash
INC 13
```

#### Remoção:

- Uma operação de remoção será identificada por uma linha escrito `REM` seguido de um espaço e depois um inteiro. Um nó com este valor deve ser removido (apenas um se houver repetição). Caso não haja um nó com o valor especificado, a estrutura não deve ser alterada. Em ambos os casos uma nova versão deve ser criada e nada precisa ser impresso.

Exemplo de linha de remoção:
```bash
REM 42
```

#### Sucessor:

- Uma operação de sucessor será identificada por uma linha escrito `SUC` seguido de um espaço, um inteiro, outro espaço e outro inteiro.

- O sucessor do número x é o menor valor na estrutura que é estritamente maior que x. O primeiro inteiro será o valor (que não precisa estar na estrutura) cujo sucessor deseja-se saber e o segundo será a versão em que o sucessor deverá ser procurado.

- Caso a versão fornecida não exista, o sucessor deve ser procurado na versão mais recente e caso não exista valor na estrutura maior que o primeiro inteiro fornecido, o sucessor será infinito. Essa operação não cria uma nova versão na estrutura e imprime a própria operação e o resultado no arquivo de saída.

Exemplo de linha de sucessor:
```bash
SUC 50 56
```

Exemplo de linha no arquivo de saída:
```bash
SUC 50 65
52
```

#### Imprimir:

- Uma operação de impressão será identificada por uma linha escrito `IMP` seguido de um espaço e um inteiro. O inteiro indica a versão da estrutura que deve ser impressa, isto é, cada elemento da estrutura na versão indicada deve ser impresso em ordem crescente seguido da sua profundidade e cor separados por vírgula.
- Caso a versão fornecida não exista, a impressão deve ocorrer na versão mais recente. Essa operação não cria uma nova versão na estrutura e imprime a própria operação e o resultado no arquivo de saída.

Exemplo de linha de impressão:
```bash
IMP 65
```

Exemplo de linha no arquivo de saída:
```bash
IMP 65
13,2,R 42,1,N 50,0,N 52,2,R 65,1,N
```

#### Versões:

- A estrutura inicialmente começa na versão 0. Cada operação de inclusão e remoção aumenta a versão da estrutura em 1. Haverá no máximo 99 operações de inclusão e remoção, de modo que haverá no máximo 100 versões diferentes da estrutura, então os identificadores das versões (raiz da estrutura e quais em quais versões ela opera) podem ser guardados num vetor de tamanho 100.
- Não há limite para o número de operações de sucessor e de impressão, mas estas não criam novas versões.

## Componentes principais

### 📁 Entrada/Saída

- Arquivo de entrada: operações do usuário (input.txt)
- Arquivo de saída: resultados (output.txt)

### ✅ Requisitos da árvore:

- Cada inclusão/remoção gera uma nova versão da árvore (persistência parcial).
- As versões devem ser mantidas num vetor Raizes[100] com ponteiros para as raízes das versões.
- Cada nó armazena:
  - Valor
  - Cor (R ou N)
  - Ponteiros para filhos e pai
  - Profundidade
  - Versão (opcional, para debug)

### 🔁 Operações principais:

- insert(valor, versão_anterior) → versão_nova
- remove(valor, versão_anterior) → versão_nova
- successor(x, versão)
- print_tree(versão)

### 🌈 Cores:

```python
# Enum para as cores dos nós
RED = 'R'
BLACK = 'N'
```

### 🧠 Persistência Parcial

- Copiar apenas os nós modificados na nova versão (copy-on-write).
- Versões antigas são preservadas.
- Estratégia: para insert ou delete, copiar o caminho até o nó modificado, com os novos nós apontando para os antigos que não mudaram.

### 📄 Estrutura dos Arquivos

`main.py`

- Função principal
- Leitura do input.txt
- Escrita no output.txt
- Controle de versões

`rbtree.py`

- Definição do Node
- Implementação das operações de árvore rubro-negra com persistência

`io.py`

- Leitura e parsing dos arquivos de entrada
- Escrita formatada no arquivo de saída

### 🧪 Testes

- Entrada com INC, REM, SUC, IMP misturados
- Versões inválidas (deve pegar a mais recente)
- Buscar sucessor de um valor maior que todos (deve retornar infinito)

🌳 Bloco 1 — Definições iniciais, Classe Node, Controle de versões, Funções Auxiliares e Funções principais.

In [1]:
# Enum para as cores dos nós
RED = 'R'
BLACK = 'N'

class Node:
    def __init__(self, key, color=RED, left=None, right=None, parent=None):
        self.key = key            # Valor do nó
        self.color = color        # 'R' ou 'N'
        self.left = left          # Filho esquerdo
        self.right = right        # Filho direito
        self.parent = parent      # Pai do nó
        self.depth = 0            # Profundidade (calculada para IMP)

    def __repr__(self):
        return f"{self.key},{self.depth},{self.color}"

# Lista com as raízes das versões da árvore (versão 0 começa vazia)
version_roots = [None] * 100  # Até 100 versões
current_version = 0

def deep_clone(node):
  if node is None:
    return None
  new_node = Node(key=node.key, color=node.color)
  new_node.left = deep_clone(node.left)
  if new_node.left:
    new_node.left.parent = new_node
  new_node.right = deep_clone(node.right)
  if new_node.right:
    new_node.right.parent = new_node
  return new_node

# ----------------- ROTAÇÕES E REBALANCEAMENTO -----------------
def rotate_left(root, node):
  right_child = node.right
  node.right = right_child.left

  if right_child.left:
    right_child.left.parent = node
  right_child.left = node

  right_child.parent = node.parent
  node.parent = right_child

  if right_child.parent is None:
    root = right_child
  elif right_child.parent.left == node:
    right_child.parent.left = right_child
  else:
    right_child.parent.right = right_child
  return root

def rotate_right(root, node):
  left_child = node.left
  node.left = left_child.right

  if left_child.right:
    left_child.right.parent = node
  left_child.right = node

  left_child.parent = node.parent
  node.parent = left_child

  if left_child.parent is None:
    root =  left_child
  elif left_child.parent.left == node:
    left_child.parent.left = left_child
  else:
    left_child.parent.right = left_child
  return root

def fix_insert(root, node):
  # enquanto o pai é vermelho, temos violação
  while node != root and node.parent.color == RED:
    parent = node.parent
    grandparent = parent.parent

    # Identificar se o pai é filho esquerdo ou direito
    if parent == grandparent.left:
      uncle = grandparent.right
      if uncle and uncle.color == RED:
        # Caso 1 – tio vermelho → recoloração
        parent.color = BLACK
        uncle.color = BLACK
        grandparent.color = RED
        node = grandparent
      else:
        # Casos 2 e 3 – tio preto → rotação
        if node == parent.right:
          node = parent
          root = rotate_left(root, node)
          parent = node.parent
          grandparent = parent.parent
        parent.color = BLACK
        grandparent.color = RED
        root = rotate_right(root, grandparent)
    else:
      uncle = grandparent.left
      if uncle and uncle.color == RED:
        parent.color = BLACK
        uncle.color = BLACK
        grandparent.color = RED
        node = grandparent
      else:
        if node == parent.left:
          node = parent
          root = rotate_right(root, node)
          parent = node.parent
          grandparent = parent.parent
        parent.color = BLACK
        grandparent.color = RED
        root = rotate_left(root, grandparent)
  root.color = BLACK
  return root

def fix_delete(root, x):

  while x and x != root and x.color == BLACK:

    parent = x.parent
    if x == parent.left:
      sibling = parent.right
      if sibling and sibling.color == RED:
        sibling.color = BLACK
        parent.color = RED
        root = rotate_left(root, parent)
        sibling = parent.right
      if (not sibling.left or sibling.left.color == BLACK) and (not sibling.right or sibling.right.color == BLACK):
        sibling.color = RED
        x = parent
      else:
        if not sibling.right or sibling.right.color == BLACK:
          if sibling.left:
            sibling.left.color = BLACK
          sibling.color = RED
          root = rotate_right(root, sibling)
          sibling = parent.right
        sibling.color = parent.color
        parent.color = BLACK
        if sibling.right:
          sibling.right.color = BLACK
        root = rotate_left(root, parent)
        x = root
    else:
      sibling = parent.left
      if sibling and sibling.color == RED:
        sibling.color = BLACK
        parent.color = RED
        root = rotate_right(root, parent)
        sibling = parent.left
      if (not sibling.left or sibling.left.color == BLACK) and (not sibling.right or sibling.right.color == BLACK):
        sibling.color = RED
        x = parent
      else:
        if not sibling.left or sibling.left.color == BLACK:
          if sibling.right:
            sibling.right.color = BLACK
          sibling.color = RED
          root = rotate_left(root, sibling)
          sibling = parent.left
        sibling.color = parent.color
        parent.color = BLACK
        if sibling.left:
          sibling.left.color = BLACK
        root = rotate_right(root, parent)
        x = root
  if x:
    x.color = BLACK
  return root

def insert(value, previous_root):
  global current_version

  # 1) Deep clone de toda a árvore anterior
  new_root = deep_clone(previous_root)

  # 2) Inserção BST no clone
  if new_root is None:
    new_node = Node(key=value, color=BLACK)
    new_root = new_node
  else:
    current = new_root
    while True:
      if value < current.key:
        if current.left:
          current = current.left
        else:
          new_node = Node(key=value)
          current.left = new_node
          new_node.parent = current
          break
      else:
        if current.right:
          current = current.right
        else:
          new_node = Node(key=value)
          current.right = new_node
          new_node.parent = current
          break

  # 3) Rebalancear a árvore rubro-negra
  new_root = fix_insert(new_root, new_node)

  # 4) Registrar nova versão
  current_version += 1
  version_roots[current_version] = new_root
  return new_root

def remove(value, previous_root):

  global current_version

  # Deep clone
  new_root = deep_clone(previous_root)

  # Exclusão de BST no clone: ​​encontrar nó
  def bst_delete(root, key):
    # A exclusão padrão retorna uma nova raiz da subárvore e do nó x para corrigir
    if root is None:
      return root, None
    if key < root.key:
      root.left, x = bst_delete(root.left, key)
      if root.left:
        root.left.parent = root
      return root, x
    elif key > root.key:
      root.right, x = bst_delete(root.right, key)
      if root.right:
        root.right.parent = root
      return root, x
    else:
      # O nó a ser excluído é raiz
      if root.left is None or root.right is None:
        child = root.left or root.right
        return child, root
      # Dois filhos: encontrar sucessor
      succ = root.right
      while succ.left:
        succ = succ.left
      root.key = succ.key
      root.right, x = bst_delete(root.right, succ.key)
      if root.right:
        root.right.parent = root
      return root, x

  new_root, x = bst_delete(new_root, value)

  if new_root and x:
    new_root = fix_delete(new_root, x)
  # Salva nova versão
  current_version += 1
  version_roots[current_version] = new_root
  return new_root

def successor(x, version_root):

  current = version_root
  succ = None

  while current:
    if current.key > x:
      succ = current
      current = current.left
    else:
      current = current.right

  return succ.key if succ else float("inf")

def print_tree(version_root):
  # Faz percurso in-order e imprime (valor, profundidade, cor)
  result = []
  def inorder(node, depth=0):
    if node:
      inorder(node.left, depth + 1)
      result.append(f"{node.key},{depth},{node.color}")
      inorder(node.right, depth + 1)
  inorder(version_root)
  return result

📄 Bloco 2 — Entrada/Saída

In [2]:
import sys

def parse_and_execute(commands):
  output_lines = []
  global current_version

  # versão 0 inicia vazia
  version_roots[0] = None

  for line in commands:
    parts = line.split()
    cmd = parts[0]

    if cmd == 'INC':
      x = int(parts[1])
      insert(x, version_roots[current_version])
    elif cmd == 'REM':
      x = int(parts[1])
      remove(x, version_roots[current_version])
    elif cmd == 'SUC':
      x, v = int(parts[1]), int(parts[2])
      if v < 0 or v > current_version or version_roots[v] is None:
        v = current_version
      succ = successor(x, version_roots[v])
      output_lines.append(f"SUC {x} {parts[2]}")
      output_lines.append(str(succ) if succ != float('inf') else 'infinito')
    elif cmd == 'IMP':
      v = int(parts[1])
      if v < 0 or v > current_version or version_roots[v] is None:
        v = current_version
      output_lines.append(f"IMP {parts[1]}")
      output_lines.append(' '.join(print_tree(version_roots[v])))
  return output_lines

if __name__ == '__main__':
  # lê de input.txt e escreve em output.txt
  commands = []
  with open('input.txt', 'r') as f:
    commands = [l.strip() for l in f if l.strip()]
  results = parse_and_execute(commands)
  with open('output.txt', 'w') as f:
    for l in results:
      f.write(l + '\n')